# Practical session 5: Core-periphery, community detection, motifs, and network filtering

In this practical session, we will learn how to find communities in networks.

We will use NetworkX version 3.5 (see [documentation](https://networkx.org/documentation/networkx-3.5/reference/index.html) and [tutorial](https://networkx.org/documentation/networkx-3.5/tutorial.html)).


#### Credits
This notebook is adapted from the tutorials associated to the book "A FIRST COURSE IN NETWORK SCIENCE" ([link to the GitHub repository](https://github.com/CambridgeUniversityPress/FirstCourseNetworkScience)). The original content is licensed under the [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

In [ ]:
from collections import Counter
from itertools import combinations
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import cpnet


print("NetworkX version: ", nx.__version__)

# Core-periphery

Let's create a core-periphery network manually and apply the core-periphery detection algorithms.

We will use the the "Core" module from NetworkX:

https://networkx.org/documentation/networkx-3.5/reference/algorithms/core.html

and the `cpnet` library:

https://github.com/skojaku/core-periphery-detection



To create a core-periphery network, we proceed as follows:
- choose the number of core nodes `n_core` and the number of periphery nodes `n_periphery`
- create a Erdos-Renyi random graph among the core nodes with a relatively high connection probability `p_core`
- create a Erdos-Renyi random graph among the periphery nodes with a relatively low connection probability `p_periphery`
- create edges between core and periphery nodes with a intermediate connection probability `p_core_periphery`

In [ ]:
RNG = np.random.RandomState(42)

# number of core and peripheral nodes
n_core = 20
n_periphery = 50

# create the core
p_core = 0.5
G_core = nx.erdos_renyi_graph(n_core, p_core)

# create the periphery
p_periphery = 0.05
G_per = nx.erdos_renyi_graph(n_periphery, p_periphery)

# relabel periphery nodes such that their names go from n_core to n_core + n_periphery - 1
# and core nodes are from 0 to n_core - 1
G_per = nx.relabel_nodes(G_per, lambda x: x + n_core)

# build the union of the two graphs
G = nx.union(G_core, G_per)

all_nodes = list(G.nodes())
core_nodes = list(range(n_core))
periphery_nodes = list(range(n_core, n_core + n_periphery))

# create edges between core and periphery
p_core_periphery = 0.15
for node in periphery_nodes:
    for core_node in core_nodes:
        if RNG.rand() < p_core_periphery:
            G.add_edge(node, core_node)

print("Network has", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges")

# draw
colors = ["red"] * n_core + ["skyblue"] * n_periphery
pos = nx.spring_layout(G, seed=42)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
nx.draw(G, pos, node_color=colors, with_labels=False, node_size=120, ax=ax)


In [ ]:
# plot adjacency matrix
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.heatmap(nx.adjacency_matrix(G).todense(), ax=ax, cmap="Blues", cbar=False)

ax.axvline(n_core, color="red", linestyle="--")
ax.axhline(n_core, color="red", linestyle="--")

ax.set_xlabel("Node index")
ax.set_ylabel("Node index")

### K-core decomposition

In [ ]:
# k-core decomposition
# dict: node -> core number
core_numbers = nx.core_number(G)

print("Core numbers:", core_numbers.values())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

# count how many times each core number appears
n_nodes_in_cores = Counter(core_numbers.values())

max_core_num = max(n_nodes_in_cores.keys())
n_nodes_in_cores = {k: n_nodes_in_cores.get(k, 0) for k in range(1, max_core_num + 1)}

ax.bar(n_nodes_in_cores.keys(), n_nodes_in_cores.values())

ax.set_xticks(range(1, max_core_num + 1))
ax.set_xlabel("Core number")
ax.set_ylabel("Number of nodes")

In [ ]:
# visualize according to k-core shell
cmap = plt.cm.viridis
norm = plt.Normalize(vmin=min(core_numbers.values()), vmax=max(core_numbers.values()))

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

nx.draw(G, pos, node_color=[cmap(norm(core_numbers[n])) for n in G.nodes()],
        with_labels=False, node_size=120, ax=ax)

ax.set_title("k-core Decomposition")


### Borgatti and Everett's method

The `cpnet` library implements a wide variety of core-periphery detection algorithms, including the original method by Borgatti and Everett (2000).
By running this method, we can obtain which nodes belong to the core and which to the periphery.

The package works as follows:
```python
algorithm = cpnet.BE(G)
node_assignment = algorithm.get_pair_id()
node_coreness = algorithm.get_coreness()
```

`node_assignment` and `node_coreness` are python dict objects that take node labels (i.e., G.nodes()) as keys.
- The values of node_assignment are integers indicating group ids: nodes having the same integer belong to the same group.
- The values of node_coreness are float values indicating coreness ranging between 0 and 1. A larger value indicates that the node is closer to the core. In case of discrete core-periphery structure, the corenss can only take 0 or 1, with node_coreness[i]=1 or =0 indicating that node i belongs to a core or a periphery, respectively.


In [ ]:
# Note: cpnet is not deterministic and seed cannot be set
niters = 50

max_score = None
best_coreperiphery = None
for _ in range(niters):

    be_cp = cpnet.BE(num_runs=10)
    be_cp.detect(G)

    # get the core-periphery assignment of nodes
    node_coreness = be_cp.get_coreness()
    node_assignment = be_cp.get_pair_id()

    # compute the loss score
    score = be_cp.score(G, node_assignment, node_coreness)

    if max_score is None or score > max_score:
        max_score = score
        best_coreperiphery = be_cp

print("Best core–periphery score:", max_score)

# now extract core and periphery nodes
node_coreness = best_coreperiphery.get_coreness()
node_assignment = best_coreperiphery.get_pair_id()


In [ ]:
# this is the assignment of coreness of each node
x = np.array([node_coreness[node] for node in G.nodes()])
x

In [ ]:
# to get the C_ij matrix, multiply x_i with x_j
C_ij = np.zeros((G.number_of_nodes(), G.number_of_nodes()))
for i in range(G.number_of_nodes()):
    for j in range(G.number_of_nodes()):
        C_ij[i, j] = x[i] * x[j]

C_ij

In [ ]:
# plot the fitted core–periphery structure
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.heatmap(C_ij, cmap="Blues", cbar=False, ax=ax)

ax.axvline(n_core, color="red", linestyle="--")
ax.axhline(n_core, color="red", linestyle="--")

# Community detection

### Partitions

A **partition** of a graph is a separation of its nodes into disjoint groups. Consider the following graph:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

G_toy = nx.Graph()
nx.add_cycle(G_toy, [0, 1, 2, 3])
nx.add_cycle(G_toy, [4, 5, 6, 7])
G_toy.add_edge(0, 7)

pos_toy = nx.spring_layout(G_toy, seed=42)
nx.draw(G_toy, pos=pos_toy, with_labels=True, ax=ax)

In [ ]:
# an example of partition
partition = [
    {1, 2, 3},
    {4, 5, 6},
    {0, 7},
]


Observe that every node in the graph is in exactly one of the sets in the partition. Formally, a partition is a list of sets such that every node is in exactly one set. NetworkX can verify that our partition is valid:

In [ ]:
# we can check using networkX functions
nx.community.is_partition(G_toy, partition)

When developing community detection algorithms, we often make use of a *partition map*, which is a dictionary mapping node names to a partition index. This is useful for quickly comparing if two nodes are in the same cluster in the partition:

In [ ]:
partition_map = {}
for idx, cluster_nodes in enumerate(partition):
    for node in cluster_nodes:
        partition_map[node] = idx

partition_map

Nodes with same partition index belong to the same community.

We can visualize our partition by drawing the graph with nodes colored by their partition membership:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

node_colors = [partition_map[n] for n in G_toy.nodes]
        
nx.draw(G_toy, pos=pos_toy, node_color=node_colors, with_labels=True, ax=ax)

Given that there is no unanimously accepted mathematical definition of community, we would like quantitative methods to partition nodes in a meaningful way.

## Community detection algorithms: Girvan-Newman method

It is an iterative algorithm whose core idea consists of, roughly, removing edges linking highly connected regions of the network. We can quantify this property using the edge betweenness centrality (similar to the node betweenness but computed on edges).

The algorithm is:
1. compute the edge betweenness centrality of all edges,
2. remove the edge with the largest centrality value (in case of ties, one of them at random),
3. assign one community to each connected component,
4. repeat steps 1-3 until all nodes are isolated

This is an example of an divisive community detection method: we start from all nodes belonging to the same commuity and we proceed until all nodes belong to different communities.

In [ ]:
def girvan_newman_communities(G, max_n_communities=None):

    # make a copy of the network
    G_copy = G.copy()

    # count number of connected components
    curr_n_conn_components = nx.number_connected_components(G_copy)
    
    # store communities found at each step
    # eg. [[{1,2,3}, {4, 5}], [{1}, {2, 3}, {4, 5}], ...]
    communities = []
    pbar = tqdm(total=G_copy.number_of_edges(), leave=False)
    while G_copy.number_of_edges() > 0:

        # compute edge betweenness and remove edge with highest betweenness
        edge_betweenness = nx.edge_betweenness_centrality(G_copy)
        edge_betweenness = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)
        edge_max_betweenness = edge_betweenness[0][0]

        G_copy.remove_edge(*edge_max_betweenness)
        pbar.update(1)

        # get the connected components
        conn_components = list(nx.connected_components(G_copy))
        n_conn_comp = len(conn_components)

        if n_conn_comp > curr_n_conn_components:
            # new communities are formed
            communities.append(conn_components)
            curr_n_conn_components = n_conn_comp

            if max_n_communities is not None and curr_n_conn_components >= max_n_communities:
                break

    # leave the progress bar
    pbar.close()

    return communities

## Application to a stochastic block model network

In [ ]:
# let's generate a planted partition with a stochastic block model
RNG = np.random.RandomState(42)

# number of nodes in each block
block_sizes = [20, 50, 30, 20]

# connection probabilities between and within blocks
# within blocks given by the diagonal entries
# between blocks by the off-diagonal entries
blocks_p = [
    [0.60, 0.30, 0.01, 0.01],
    [0.30, 0.50, 0.01, 0.01],
    [0.01, 0.01, 0.40, 0.01],
    [0.01, 0.01, 0.01, 0.50],
]

G = nx.stochastic_block_model(block_sizes, p=blocks_p,  seed=RNG, directed=False, selfloops=False)

In [ ]:
# show the block connection probabilities
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

sns.heatmap(blocks_p, vmin=0, vmax=0.5, cmap="Blues", cbar=True, ax=ax)

ax.set_xlabel("Block index")
ax.set_ylabel("Block index")

In [ ]:
# draw the network
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

pos = nx.spring_layout(G, seed=42)
node_colors = [attr['block'] for _, attr in G.nodes(data=True)]

nx.draw(G, pos=pos, node_size=50, node_color=node_colors, ax=ax)

How many communities do you expect to find?

Let's apply the Girvan-Newman method to the network.

In [ ]:
# based on our implementation, it returns all the community splits found
communities_gn_list = girvan_newman_communities(G, max_n_communities=10)

Let's visualize one of such partitions.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

# make the partition map
choosen_gn_partition = communities_gn_list[4]

print("Number of communities in the chosen partition:", len(choosen_gn_partition))

partition_map_1 = {}
for idx, cluster_nodes in enumerate(choosen_gn_partition):
    for node in cluster_nodes:
        partition_map_1[node] = idx

# plot the network and compute modularity
node_colors = [partition_map_1[n] for n in G.nodes]  
nx.draw(G, pos=pos, node_color=node_colors, node_size=50, ax=ax)

mod_1 = nx.community.modularity(G, choosen_gn_partition)
ax.set_title(f"Q = {mod_1:.4f}")

How can we evaluate the quality of a partition?

One of the first proposals is the *modularity* (introduced in 2004):
$$ Q = \frac{1}{2|E|} \sum_{i,j\ne i}^N \left(A_{i,j} - \frac{k_i k_j}{2|E|} \right) \delta\left(C_i, C_j \right) $$

The modularity of a graph partition compares the number of intra-group edges with a random baseline (i.e., a configuration model). Higher modularity scores correspond to a higher proportion of intra-group edges, therefore fewer inter-group edges and better separation of groups.

NetworkX has a function to compute the modularity of a partition: `nx.community.modularity` 

In [ ]:
# compute the modularity of the obtained partitions
Qs_gn = [nx.algorithms.community.quality.modularity(G, comm) for comm in communities_gn_list]
n_communities_gn_list = [len(comm) for comm in communities_gn_list]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

ax.plot(n_communities_gn_list, Qs_gn, marker='o')

# highlight the maximum
idx_max = np.argmax(Qs_gn)
n_communities_opt = n_communities_gn_list[idx_max]

ax.plot(n_communities_opt, Qs_gn[idx_max], marker='o', color='red', markersize=10)

ax.set_xlabel("Number of communities")
ax.set_ylabel("Modularity Q")

Let's now compare the similarity between the optimal partition and the partition from which the network was generated.

One possible measure is the Jaccard index:
$$ J = \frac{a_{11}}{a_{01} + a_{10} + a_{11}} $$

where: 
- $a_{11}$ is the number of pairs of nodes that are in the same community in both partitions,
- $a_{01}$ is the number of pairs of nodes that are in the same community in the first partition but not in the second,
- $a_{10}$ is the number of pairs of nodes that are in the same community in the second partition but not in the first.

In [ ]:
def flatten_partition(part):
    pairs_of_same_comm_nodes = set()
    for comm in part:
        for pair in combinations(comm, 2):
            pair = tuple(sorted(pair))
            pairs_of_same_comm_nodes.add(pair)

    return pairs_of_same_comm_nodes


def jaccard_index(part1, part2):

    part1_flat = flatten_partition(part1)
    part2_flat = flatten_partition(part2)

    inters = len(set(part1_flat).intersection(set(part2_flat)))
    union = len(set(part1_flat).union(set(part2_flat)))

    return inters / union


In [ ]:
# get the true partition
part_true = [list(range(sum(block_sizes[:i]), sum(block_sizes[:i+1]))) for i in range(len(block_sizes))]

In [ ]:
idx_max = np.argmax(Qs_gn)
best_partition_gn = communities_gn_list[idx_max]

jaccard_index(part_true, best_partition_gn)

# Disparity filter

In [ ]:
# load the ecological food web network seen in class
# a directed link from i to j means that species i is preyed upon by species j
G = nx.read_weighted_edgelist("../datasets/FloridaFoodWeb/edges.csv", create_using=nx.DiGraph, delimiter=",")

# get largest weakly connected component
largest_wcc = max(nx.weakly_connected_components(G), key=len)
G = G.subgraph(largest_wcc).copy()

# remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))

G.number_of_nodes(), G.number_of_edges()

In [ ]:
# define disparity filter method
def compute_disparity_filter_probas(G):

    # create a edge attribute that corresponds to the p_ij
    edge_probas = {}
    for node in G.nodes():

        # triplets (i, j, w)
        node_edges = G.edges(node, data='weight')
        k = len(node_edges)

        # compute the strength
        strength = sum([w for _, _, w in node_edges])

        for u, v, w in node_edges:
            p_ij = (1 - w / strength) ** (k - 1)
            edge_probas[(u, v)] = p_ij

    if not G.is_directed():
        # in general p_ij != p_ji
        # for this reason, in undirected networks, we consider that an edge is significant if either p_ij or p_ji is smaller than alpha
        # thus, we impose that p_ij = min(p_ij, p_ji)
        for (u, v), p_ij in edge_probas.items():

            if G.has_edge(v, u):
                p_ij_min = min(edge_probas[(u, v)], edge_probas[(v, u)])
                edge_probas[(u, v)] = p_ij_min
                edge_probas[(v, u)] = p_ij_min

    return edge_probas

# set the new edge attribute
edge_probas = compute_disparity_filter_probas(G)
nx.set_edge_attributes(G, edge_probas, name='p_ij')

In [ ]:
def disparity_filter(G, p_ij_attr, alpha):
    G_filtered = G.copy()
    for edge in G.edges():
        if G.edges[edge][p_ij_attr] >= alpha:
            G_filtered.remove_edge(edge[0], edge[1])

    # remove isolated nodes
    isolated_nodes = list(nx.isolates(G_filtered))
    G_filtered.remove_nodes_from(isolated_nodes)
    
    return G_filtered


# define thresholding function
def threshold_filter(G, threshold):
    G_filtered = G.copy()
    for edge in G.edges():
        if G.edges[edge]['weight'] <= threshold:
            G_filtered.remove_edge(edge[0], edge[1])

    # remove isolated nodes
    isolated_nodes = list(nx.isolates(G_filtered))
    G_filtered.remove_nodes_from(isolated_nodes)
    
    return G_filtered

In [ ]:
# run the naive threshold filter using different values the the weight as a threshold
all_edge_weights = [w for i, j, w in G.edges(data='weight')]
min_threshold = min(all_edge_weights)
max_threshold = max(all_edge_weights)

thresholds = np.logspace(np.log10(min_threshold), np.log10(max_threshold), 500)

results_threshold_df = []
for threshold in tqdm(thresholds):

    G_filtered = threshold_filter(G, threshold)

    # get the number of nodes and edges of the filtered graph
    num_nodes_retained = G_filtered.number_of_nodes()
    num_edges_retained = G_filtered.number_of_edges()
    
    results_threshold_df.append([threshold, num_nodes_retained, num_edges_retained])

results_threshold_df = pd.DataFrame(results_threshold_df, columns=['threshold', 'num_nodes', 'num_edges'])
results_threshold_df.head()

In [ ]:
# run the disparity filter using different values of alpha
alphas = np.logspace(-6, 0, 100)

results_disparity_df = []
for alpha in tqdm(alphas):
    
    G_filtered = disparity_filter(G, 'p_ij', alpha)

    # remove isolated nodes
    isolated_nodes = list(nx.isolates(G_filtered))
    G_filtered.remove_nodes_from(isolated_nodes)

    # get the number of nodes and edges of the filtered graph
    num_nodes_retained = G_filtered.number_of_nodes()
    num_edges_retained = G_filtered.number_of_edges()

    results_disparity_df.append([alpha, num_nodes_retained, num_edges_retained])

results_disparity_df = pd.DataFrame(results_disparity_df, columns=['alpha', 'num_nodes', 'num_edges'])
results_disparity_df.head()

In [ ]:
# compute the fraction of retained nodes and edges
N, M = G.number_of_nodes(), G.number_of_edges()

results_threshold_df.loc[:, 'frac_retained_nodes'] = results_threshold_df['num_nodes'] / N
results_threshold_df.loc[:, 'frac_retained_edges'] = results_threshold_df['num_edges'] / M

results_disparity_df.loc[:, 'frac_retained_nodes'] = results_disparity_df['num_nodes'] / N
results_disparity_df.loc[:, 'frac_retained_edges'] = results_disparity_df['num_edges'] / M

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.plot(results_threshold_df['frac_retained_edges'], results_threshold_df['frac_retained_nodes'], "o-", label='Thresholding')
ax.plot(results_disparity_df['frac_retained_edges'], results_disparity_df['frac_retained_nodes'], "o-", label='Disparity Filter')

ax.set_xlabel('Fraction of retained edges')
ax.set_ylabel('Fraction of retained nodes')
ax.legend()

Let's look at the graph from right (i.e., the original graph) to left (graph completely deleted). The disparity filter is able retain all the nodes while removing most part of edges (>80%). In comparison, the threshold filter removes ~40% of the nodes to achieve a similar reduction of number of edges. In conclusion, by looking at the "significant" connection, we can reduce the number of edges while retaining a larger fraction of nodes. For the analysis of this system, the disparity filter would be preferable because it retains almost all elements of the system (i.e., nodes) while removing non-significant connections between them.

# Motifs

In [ ]:
# load the Florida food web dataset
G = nx.read_weighted_edgelist("../datasets/FloridaFoodWeb/edges.csv", create_using=nx.DiGraph, delimiter=",")

# get largest weakly connected component
largest_wcc = max(nx.weakly_connected_components(G), key=len)
G = G.subgraph(largest_wcc).copy()

# remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))

G.number_of_nodes(), G.number_of_edges()

In [ ]:
# count the number of all possible triads
# see explanation of triad names in: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.triads.triads_by_type.html#networkx.algorithms.triads.triads_by_type
nx.triadic_census(G)

However, are those three-node motifs occurring just by chance? Or there is a biological reason for their presence?

To answer this question, we can compare the motif counts with those obtained from a null model. A common choice is the configuration model, which preserves the in-degree and out-degree of each node while randomizing the connections.

Once we have generated multiple instances of the configuration model, we can compute the motif counts in each instance and compare them with the original graph using the z-score:
$$ z = \frac{N_{real} - \mu_{null}}{\sigma_{null}} $$
where $N_{real}$ is the motif count in the real graph, and $\mu_{null}$ and $\sigma_{null}$ are the mean and standard deviation of the motif counts in the null model instances.

In [ ]:
# count the occurrence of each triad in random graphs generated by the directed configuration model
# repeat 50 times
RNG = np.random.RandomState(42)

niters = 50
triad_census_rand_df = []
for _ in tqdm(range(niters), total=niters, leave=False):

    G_rand = nx.directed_configuration_model(
        [d for n, d in G.in_degree()],
        [d for n, d in G.out_degree()],
        create_using=nx.DiGraph, 
        seed=RNG)
    
    triad_census_rand_ = nx.triadic_census(G_rand)
    triad_census_rand_df.append(triad_census_rand_)

# append also the real one - column niters is the one containing the real counts
triad_census_real = nx.triadic_census(G)
triad_census_rand_df.append(triad_census_real)

triad_census_rand_df = pd.DataFrame(triad_census_rand_df).T
triad_census_rand_df.head()

In [ ]:
# compute the z-scores

# compute mean and std occurrence of each motif in the random graphs (columns from 0 to niters-1)
triad_census_rand_df.loc[:, "avg"] = triad_census_rand_df[0:niters].mean(axis=1)
triad_census_rand_df.loc[:, "std"] = triad_census_rand_df[0:niters].std(axis=1)

# compute the z-score - the niters column contains the real counts
triad_census_rand_df.loc[:, "z-score"] = (triad_census_rand_df[niters] - triad_census_rand_df["avg"]) / triad_census_rand_df["std"]
triad_census_rand_df.sort_values("z-score", inplace=True)

# take only relevant columns
triad_census_rand_df = triad_census_rand_df[["avg", "std", "z-score"]]

triad_census_rand_df.head()

In [ ]:
# plot the motif profiles
fig, ax = plt.subplots(1, 1, figsize=(12, 3))

ax.plot(triad_census_rand_df["z-score"], "o")

ax.axhline(0, color="gray", linestyle="--")

ax.set_xlabel("Triad types")
ax.set_ylabel("z-score")

003 : 1, 2, 3

030C : 1 <- 2 <- 3, 1 -> 3

111U : 1 <-> 2 -> 3

300 : 1 <-> 2 <-> 3, 1 <-> 3

021D : 1 <- 2 -> 3

030T : 1 -> 2 -> 3, 1 -> 3

021U : 1 -> 2 <- 3

What is your interpretation given the nature of the network?